# Predicting Sales Rank for a Product on Amazon using Product Description

#### DATA 200, Spring 2021
#### Final Project
#### Project members:
- Alexander Wu, 3032676584, alexwu68@berkeley.edu
- Jonathan Kupfer, 25718319, jkupfer@berkeley.edu
- Utkarsh Yadav, 3035277597, utkarsh_yadav@berkeley.edu



## Introduction
This project aims to use Amazon metadata (e.g., product description, price, brand) to predict the Amazon sales rank within a specific category of data. The categories this project focuses on are All Beauty products. 

## Motivation
Our target for this project is new venders who are interested in adding a product to Amazon. This model is helpful for them to understand what information (e.g., keywords in the description, specific brands/brand recognition, price, etc) can be used to improve a product's sales rank within its category.

## Research Question
How can the metadata of Amazon products for a given category be used to predict a new product's sales rank in that category?

## Literature Review
While we have not found this specific research question answered in existing literature, other projects use similar datasets to make sales-related predictions. A blog AIHello used deep learning to predict a product's sales, given the product's sales rank (AIHello, 2019). Because they focused on top sellers and top seller categories, they were not able to make predictions for less high-selling categories. Numerous academic papers have researched sales predictions at Amazon. One paper by Singh et al. titled 'Sales Forecast for Amazon Sales with Time Series Modeling' uses neural networks to forecast future sales at Amazon using historical sales data. These predictions were not category-specific. This project adds to the field by showing how you can predict a product's sales rank within its own category using the metadata for a product, specifically using many features that the vendor has control over (Singh, 2020).  



## Methodolgy

### Description of the data:
This report studies two datasets from Amazon sales. The reviews datasets and the metadata dataset. This study is on All Beauty products on Amazon. The attributes of the metadata dataset along with a short description for each attribute is provided below:
- `asin` - ID of the product, e.g. 0000031852
- `title` - name of the product
- `feature` - bullet-point format features of the product
- `description` - description of the product
- `price` - price in US dollars (at time of crawl)
- `image` - url of the product image
- `related` - related products (also bought, also viewed, bought together, buy after viewing)
- `salesRank` - sales rank information
- `brand` - brand name
- `categories` - list of categories the product belongs to
- `tech1` - the first technical detail table of the product
- `tech2` - the second technical detail table of the product
- `similar` - similar product table

Similarly the attributes of the reviews dataset:
- `reviewerID` - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- `asin` - ID of the product, e.g. 0000013714
- `reviewerName` - name of the reviewer
- `vote` - helpful votes of the review
- `style` - a disctionary of the product metadata, e.g., "Format" is "Hardcover"
- `reviewText` - text of the review
- `overall` - rating of the product
- `summary` - summary of the review
- `unixReviewTime` - time of the review (unix time)
- `reviewTime` - time of the review (raw)
- `image` - images that users post after they have received the product


### Exploratory Data Analysis
#### Missing prices:
### Description of methods:
#### Feature Engineering
- One Hot Encoding: We chose to use a one hot encoding of the descriptions because... 
- Price: Price is used as a feature in our model. The vendor can set the price of a product, and the price will affect a buyer's willingness to pay for the product.
- Price$^2$: Price squared is used as a feature in our model. Having the price squared will weight higher-priced items more heavily, which could negatively affect sales-rank. 
- Brand Counts: This feature represents the number of total products of the same brand in the dataset. This could be a proxy for brand recognition, but also for products that are more ubiquitous. 
- Brand$^2$: Brand squared is used as a feature in our model to highlight more ubiquitous brands. 

By squaring the price and brand count, we are allowing the model to train on a non-linear dataset. 



The square root of the sales rank was used as the label, because the distribution of the sales rank is more symmetric after this transformation than originally or when transformed with a logarithmic function, so we believed it would be easier for a linear model to predict.

#### Modeling

The above features and label were used to train three machine learning models. The training data consisted of a random 80% of data points and the rest were used for validation.

- Ridge Regression: The `sklearn` class `RidgeCV` was used to perform $l_2$ regularized linear regression.
    - This model was used to investigate whether a simple linear model would perform well. Once it was shown that a support vector machine performed better, the nonlinear features price squared and brand count squared were added to reduce the model bias.
    - The `sklearn` built-in cross validation was used over several alpha values, in particular `np.exp(np.arange(-5, 5, 0.2) * np.log(10))`.
- Soft Margin Support Vector Machine: The `sklearn` class `SVR` was used to to train the data on a regularized SVM.
    - SVMs were used because the ridge regression model exhibited high bias, so we believed a nonlinear kernel might enable the model to make better predictions.
    - We compared the Gaussian, polynomial, linear, and sigmoid kernels. For each kernel, we performed manual regularization on the test $r^2$ of the model sales rank and chose the best $C$ hyperparameter.
- Vanilla Random Forest: The `sklearn` class `RandomForestRegressor` was used to train a random forest.
    - Because ridge regression and SVMs still had relatively high bias, we used tried using a random forest. Individual decision trees have low bias, so they allowed for an expressive model, but averaging over many decision trees would keep the variance relatively low. In addition, they would allow a nonlinear model, even more so than squared features and nonlinear kernels.
    - Manual regularlization using the test `r^2` allowed us to adjust the three hyperparameters, the number of estimators, maximum tree depth, and minimum leaf size.

## Summary of results:

### Missing prices:
The metadata dataset for the All Beauty category has several products with missing prices. This initially came as a surprise and is also concerning as price is an important feature in our model. About 65% of the products were missing prices. 
It was found that a lot of these were products that are no longer sold on Amazon and are now replaced by their better/improved versions.

Certain visualizations can help us explore this further. The figure below shows a scatter plot between the proportions of products missing prices for a brand vs the total products that brand ever sold. It is important to note that the X axis is not the total sales but the total products by the brand that are listed on Amazon.


![EDA1](../figures/EDA_Prices_2.png)


But this probably includes a lot of brands which only ever sold a few products and shut down(notice so many scatter dots at y =1 close to x=0). If we only include brands which have atleast a few products with active prices on Amazon we see a very positive correlation between Y and X.


![EDA2](../figures/EDA_Prices_2.png)

Figure above shows how bigger brands in the industr industry keep improving and releasing new/revised products to survive the Amazon marketplace. It is astonishing how quickly the inventory has to evolve.

### Sentiment of reviews:
Sentiment itself is not a feature of this model but this analysis was done to see how the sentiment varies with the ratings given for products.
To get the polarity of the review text, VADER was used. 
As a part of the data cleaning process, all the punctuation was removed and the text was converted to lower case. Once extracted, we first observe polarity vs overall rating in each review. 

![EDA3](../figures/EDA_Sentiment_1.png)

Overall, it seems there is a positive correlation between polarity and the rating as one would expect. We further explore this by differentiating the reviews over verified and unverified users.

![EDA4](../figures/EDA_Sentiment_2.png)

It seems that unverified users have a wider spread in the polarity at every rating. This is mainly because the number of verified users far exceeds the number of unverified users in this category. One should also note that a lot of reviews are bound to have spelling mistakes which would not be recognized by VADER.


### Description of methods:
#### Feature Engineering
The following features are included in our model:
- Price: Price is used as a feature in our model. The vendor can set the price of a product, and the price will affect a buyer's willingness to pay for the product.
- Price$^2$: Price squared is used as a feature in our model. Having the price squared will weight higher-priced items more heavily, which could negatively affect sales-rank. 
- Brand Counts: This feature represents the number of total products of the same brand in the dataset. This could be a proxy for brand recognition, but also for products that are more ubiquitous. 
- Brand$^2$: Brand squared is used as a feature in our model to highlight more ubiquitous brands. 
- Whether the brand is considered a _top brand_: if the brand appears more than 20 times in the test set.
- Top 100 bag of words included in the title
- Top 500 bag of words included in the descriptions
- One hot encoding of brand
By squaring the price and brand count, we are allowing the model to train on a non-linear dataset. Furthermore, we standardize all numerical features included in our model (brand count, brand count squared, price, and price squared.



### Model Predictions

The model results are summarized in the following table. The table includes
- The training and test $r^2$
- The training and test loss, calculated as $\sqrt{\frac1n\sum_{i=1}^n(y_i-\hat y_i)^2}$
- The training and test proportional loss, calculated as $\sqrt{\frac1n\sum_{i=1}^n\left(\frac{y_i-\hat y_i}{y_i}\right)^2}$
- The optimal hyperparameters found through manual validation using test $r^2$

| Model                  | Training $r^2$ | Test $r^2$ | Training loss | Test loss | Training proportional loss | Test proportional loss | Optimal hyperparameters |
|------------------------|--------------|----------|---------------|-----------|----------------------------|------------------------|-------------------------|
| Ridge Regression       | 0.1290       | 0.0506   | 342.1         | 346.9     | 1.660                      | 2.881                  | N/A                     |
| Support Vector Machine | 0.2590       | 0.1171   | 315.6         | 334.6     | 1.287                      | 2.020                  | $C=250$                 |
| Random Forest          | 0.5099       | 0.1190   | 256.6         | 334.2     | 0.874                      | 1.899                  | n_estimators=100, max_depth=27, min_samples_leaf=1 |

For reference, the label standard deviation is $364.5$.

## Discussion:
Analysis of your findings to answer your research question(s). Include visualizations and specific results. If your research questions contain a modeling component, you must compare the results using different inference or prediction methods (e.g., linear regression, logistic regression, or classification and regression trees). Can you explain why some methods performed better than others?


### Model Accuracy

#### Comparison Between Models

Based on test $r^2$, vanilla random forest performed best, with SVM only slightly worse, while ridge regression, even with some nonlinear features, performed significantly worse. This is likely because both numerical features and one hot features combine in complex nonlinear ways to influence the sales rank, so two simple squared features were not enough to capture the relationship.

The random forest model had significantly higher training $r^2$ than the other models, despite its test $r^2$ being barely better than SVM. This is likely because with large maximum tree depth and small minimum node size, the random forest still has high variance despite the ensemble smoothing the prediction. However, the variance is low enough to give the model the best test result.

Despite having similar test $r^2$ to the SVM, the random forest model has lower average test proportional loss, which suggests that it is more accurate for products with small sales rank than the other models. Thus it may be a better model than its $r^2$ implies. The reason for this might be because it can utilize step functions that bypass restrictions of linear functions.

We also graphed the proportional residuals for each model, equal to the residuals divided by the correct label value. See the figures below:

Training data | Test data
:-------------------------:|:-------------------------:
![r1](../figures/ridge_residuals_train.png) | ![r2](../figures/ridge_residuals_test.png)
![s1](../figures/svm_residuals_train.png) | ![s2](../figures/svm_residuals_test.png)
![rf1](../figures/rf_residuals_train.png) | ![rf2](../figures/rf_residuals_test.png)

Interestingly, the SVM contains a large spike of training points with practically zero error. However, this is no longer true for its test points, so this is not a significant advantage. Other than this, the models have a fairly similar and reasonable spread of residuals.

#### Evaluation of Practicality

All three models had test $r^2$ values significantly lower than the training $r^2$, which suggests they are all overfitting.

All three $r^2$ values are relatively low, no more than $0.12$. We suspect that this is because the sales rank simply cannot be accurately predicted with the metadata we supplied to the models. This is reasonable because the sales performance of a product depends on more than its listing on the Amazon website. The physical quality and accompanying service of the product likely needs to be taken into account to better predict how it will perform. This is acceptible, since we are hoping to roughly predict how a product will perform based on its listing alone. With our models, we have been able to explain slightly more than 10% of the variance in product sales performance, which is enough to inform users of the Amazon listing service about how their products will perform, before the product is launched on the site.

To get a more concrete idea of how the models are performing, we listed the first ten predictions on the test set:

| Product                                                                                                                                   | Sales rank | Ridge prediction | SVM prediction | Random forest prediction |
|-------------------------------------------------------------------------------------------------------------------------------------------|------------|------------------|----------------|--------------------------|
| MICA Beauty Waterproof Line Fix Eyeliner Gel Pencil in Black                                                                              | 399        | 822              | 810            | 712                      |
| Moon Juice - Organic Power Dust \| Edible Strength (1.5 oz)                                                                               | 358        | 843              | 861            | 783                      |
| LA MER MOISTURIZING CREAM BY LA MER FOR WOMEN 1.0 OZ  *SEALED*                                                                            | 929        | 874              | 733            | 929                      |
| Victoria's Secret Beauty Rush Lip Stain Gloss Love Always                                                                                 | 1356       | 894              | 925            | 990                      |
| SEPHORA+PANTONE UNIVERSE Color of the Year Layer Lipstick                                                                                 | 729        | 947              | 950            | 939                      |
| LG G4 Case, AutumnFall&reg; Premium Slim Flexible Soft Bumper PC Protective Case Cover for LG G4 (C)                                      | 892        | 1078             | 1115           | 1107                     |
| HuntGold Butt Station Desk Office Tape Pen Paper Clip Holder Organizer Stand Blue                                                         | 1420       | 924              | 928            | 921                      |
| 1 Bottle Go Hair Silky Seaweed Nutrients Deep Penetrating Hair Treatment 250ml[Free Ceramine UV Line Ginkgo Plus Whitening Cream 8.50ml.] | 786        | 872              | 878            | 804                      |
| Lookatool&reg; 4Pcs Makeup Cosmetic Tool Eyeshadow Powder Foundation Blending Brush Set                                                   | 479        | 614              | 623            | 800                      |
| Whats Up Nails - Heartbeat Tape Vinyl Stencils for Nail Art Design (1 Sheet, 20 Stencils)                                                 | 302        | 858              | 864            | 920                      |

As we can see, all three models have some bias that causes them to tend to predict values close to each other, roughly between 700 and 1100, even though many products have square root sales ranks outside of that range. The three models have varying accuracies on different data points as well. However, these model are still relatively close to each other. We expect they are capturing the aspects of the product that can be inferred from the listing description on Amazon, while the real sales rank also depends on the actual quality of the product. Therefore, if a merchant is to use our model, it should be emphasized that the model output is not an accurate prediction of how the product will perform, but simply a useful piece of information about the quality of the site listing.

## Limitations:
An evaluation of your approach and discuss any limitations of the methods you used.

### Model Predictions

As mentioned earlier, the model is not able to accurately predict the sales rank without more data about the physical quality of the product, even if it is useful for evaluating a product listing alone.

All three models used here were relatively simple. If we wished to find a model with even lower bias, it would be worth investigating whether a deep neural network could produce even lower test loss.

We used the test $r^2$ as the main metric to judge our models, with proportional test loss as a secondary consideration. This is because it is easier to train this model, and a difference of one rank place is not necessarily more important for small ranks than large ranks. However, if one is concerned about over-emphasizing large ranks, we could adjust the models to use a different metric.

## Surprises and future work:
The following are a some of the things that surprised us while working on this project:
- Missing Data: A lot of the data had missing categories or required a lot of cleaning. Sales rank, for example, was not given to us as an integer but rather a long string that included the specific category that it was a part of. Price, a feature that one would expect for every product, was missing for many of the products. Working with this real-world data, we saw how much cleaning was needed to get the data into a workable form to build a model with.
- Predicting valuable information: After choosing which dataset to work on, we were very torn what we wanted to predict: whether we wanted to focus on sales rank, the number of stars, recommending similar products, or something else.


As mentioned in the Limitations section, there are several ways we could further investigate the modeling:

- We could look into the unused features that were not incorporated due to their difficulty in analysis, such as image and technical description.

- We could try using a deep neural network to train on the data.

- We could use a different metric other than the $r^2$ coefficient to train and validate the models.

## Works Cited:
- B. Singh, P. Kumar, N. Sharma and K. P. Sharma, 'Sales Forecast for Amazon Sales with Time Series Modeling,' 2020 First International Conference on Power, Control and Computing Technologies (ICPC2T), 2020, pp. 38-43, doi: 10.1109/ICPC2T48082.2020.9071463.
- AIHello, 'Predicting Amazon sales using Deep Learning,' 2019  AIHello.com, Retrieved from: https://www.aihello.com/resources/blog/2020/04/13/predicting-amazon-sales-using-deep-learning/.

We performed the analysis with the All Beauty products category. The same analysis can be done with other categories to discover whether they behave differently.

As mentioned in the Limitations section, we could try training a neural network to further improve model accuracy. Or, we could use some proportional metric to place less emphasis on large label values.

In this project we attempted to predict the sales rank because it is likely more important to a merchant on Amazon. However, the average rating of a product is quite important as well, and moreover, the rating is often not strongly correlated with the sales rank. A future project could look into predicting product ratings as well.